In [249]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import linear_model
from sklearn.feature_selection import mutual_info_regression, RFE, mutual_info_classif, VarianceThreshold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from rmse import compute_rmse

In [250]:
X1 = pd.read_csv("X1.csv")
Y1 = pd.read_csv("Y1.csv", header=None, names=["shares"])

In [251]:
n_samples, n_feats = X1.shape
X1.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,10.0,379.0,0.521622,1.0,0.697248,5.0,2.0,1.0,0.0,4.126649,...,0.368452,0.1,0.5,-0.244444,-0.500000,-0.050000,0.227273,0.068182,0.272727,0.068182
1,12.0,402.0,0.502660,1.0,0.645299,12.0,10.0,1.0,0.0,4.537313,...,0.270170,0.1,0.5,-0.159259,-0.166667,-0.155556,0.500000,0.500000,0.000000,0.500000
2,11.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
3,9.0,256.0,0.619608,1.0,0.792208,2.0,0.0,0.0,1.0,4.644531,...,0.245130,0.1,1.0,-0.500000,-0.500000,-0.500000,0.200000,0.100000,0.300000,0.100000
4,12.0,268.0,0.591603,1.0,0.740260,1.0,1.0,1.0,1.0,4.671642,...,0.539076,0.2,1.0,-0.254167,-0.500000,-0.050000,0.000000,0.000000,0.500000,0.000000


In [252]:
Y1.head()

,shares
0,1200
1,2500
2,1500
3,1800
4,542


# Feature selection : filter approach first

In [253]:
features = [i for i in X1.keys()]
print(features)

['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length', 'num_keywords', 'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words', 'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarit

In [257]:
# 'is_weekend' is a linear combination of 'weekday_is_saturday'and 'weekday_is_sunday'
weekend = X1["weekday_is_saturday"] + X1["weekday_is_sunday"]
corr = weekend.corr(X1["is_weekend"])
print(corr)

1.0


In [255]:
data_channel = X1['data_channel_is_lifestyle'] + X1['data_channel_is_entertainment'] + X1['data_channel_is_bus'] + X1['data_channel_is_socmed'] + X1['data_channel_is_tech'] + X1['data_channel_is_world']

In [233]:
# correlation matrix between features
corr_matrix = X1.corr()

In [234]:
for i in corr_matrix.keys():
    for j,k in corr_matrix[i].items():
        if corr_matrix[i][j] >= 0.95 and i != j:
            print(i,j)

kw_max_min kw_avg_min
kw_avg_min kw_max_min


In [236]:
# use this because Y is discrete variable -> https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html
MI = mutual_info_classif(X1,Y1)
print(MI)

[2.58678419e-02 1.17225235e-02 3.34840782e-03 5.35212849e-01
 4.53411481e-04 1.35717037e-02 1.86387101e-02 1.45842979e-01
 2.14124148e-02 1.06653704e-02 1.64983731e-02 0.00000000e+00
 1.90888702e-02 6.03751400e-03 1.41546274e-02 1.39240276e-02
 3.28255098e-02 1.98881254e-01 1.13804595e-02 1.24173948e-02
 9.93914062e-03 3.14349724e-01 1.63273865e-02 3.40868875e-02
 3.81972631e-02 3.19302477e-02 2.73285894e-02 3.05848757e-02
 4.71866389e-02 0.00000000e+00 2.51306672e-03 0.00000000e+00
 1.04838313e-02 8.10350322e-03 1.46875951e-02 5.95259235e-03
 2.23268494e-02 1.96653788e-02 1.48366299e-02 2.22797998e-02
 2.13947575e-02 2.54365745e-02 4.57158385e-03 7.64721799e-03
 1.61022550e-03 0.00000000e+00 2.34161301e-04 1.59304492e-03
 2.84010319e-03 8.11997184e-02 5.33809805e-02 0.00000000e+00
 9.56994611e-03 2.67674835e-02 1.22014884e-02 5.50872884e-04
 1.40945922e-01 2.88561893e-03]


In [238]:
features_0_MI = [(features[i],i) for i in range(len(MI)) if MI[i] == 0]
print(features_0_MI)

[('data_channel_is_lifestyle', 11), ('weekday_is_monday', 29), ('weekday_is_wednesday', 31), ('global_rate_negative_words', 45), ('avg_negative_polarity', 51)]


In [239]:
# remove features that have 0 mutual info
for i in features_0_MI:
    features.pop(i[1])
    X1 = X1.drop(i[0],1)
X1.shape

(19822, 53)

In [240]:
print(len(features))

53


In [241]:
sel = VarianceThreshold() # no parameter so remove all zero variance features
sel.fit_transform(X1)
print("Number of features with zero variance: " + str(X1.shape[1]-len(sel.variances_)))
sel.variances_

Number of features with zero variance: 0


array([4.46787876e+00, 8.47400000e+03, 1.86280137e-02, 2.85467125e-02,
       2.34718086e-02, 1.23272729e+02, 1.56426224e+01, 6.93468052e+01,
       1.60219057e+01, 7.03784556e-01, 3.67005846e+00, 1.47794923e-01,
       1.34107534e-01, 5.52297574e-02, 1.51058618e-01, 1.64087309e-01,
       3.78000000e+02, 2.98400000e+05, 4.28288571e+04, 8.43300000e+05,
       8.43300000e+05, 8.43300000e+05, 3.61071838e+03, 2.98400000e+05,
       4.35676599e+04, 6.90400000e+05, 8.43300000e+05, 6.90400000e+05,
       1.52008450e-01, 1.49689203e-01, 1.23286700e-01, 5.76711585e-02,
       6.50331793e-02, 1.14111323e-01, 6.95220164e-02, 4.84788179e-02,
       7.83866948e-02, 8.73689942e-02, 8.44382378e-02, 1.35781671e-02,
       9.31495379e-03, 3.02938832e-04, 3.58479204e-02, 2.42453788e-02,
       1.08967107e-02, 4.93520885e-03, 6.14883854e-02, 8.36385941e-02,
       9.38918552e-03, 1.05307072e-01, 6.97871978e-02, 3.56897836e-02,
       5.05401104e-02])

# Linear regression

In [244]:
regr = linear_model.LinearRegression()
predict = regr.fit(X1,Y1)

In [247]:
lm = regr.intercept_
for i in range(0,52):
    lm = lm + predict.coef_[0][i] * X1[features[i]]

KeyError: 'weekday_is_monday'

In [91]:
X1[features[0]]

0        10.0
1        12.0
2        11.0
3         9.0
4        12.0
         ... 
19817     8.0
19818    11.0
19819    14.0
19820    10.0
19821    13.0
Name: n_tokens_title, Length: 19822, dtype: float64